In [1]:
import pandas as pd

file_paths = {
    "nonhate": "transcripts/transcriptions_OWN_NON_HATE_VIDEOS.csv",
    "explicithate": "transcripts/transcriptions_OWN_EXPLICIT_HATE_VIDEOS.csv",
    "implicithate": "transcripts/transcriptions_OWN_IMPLICIT_HATE_VIDEOS.csv"
}

dfs = []
for label, path in file_paths.items():
    df = pd.read_csv(path)
    df["label"] = label
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df = df.drop(columns=["file_name"])

# Save to CSV if needed
df.to_csv("transcripts/all_transcriptions.csv", index=False)

In [2]:
import os
import time

import openai
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


def moderate_df(df, text_column="Transcription", batch_size=32, only_na=True):
    # Select rows to process (only NA values if only_na=True)
    df_to_process = df[df["prediction"].isna()] if only_na else df

    for i in range(0, len(df_to_process), batch_size):
        print(i, end="-")
        batch = df_to_process.iloc[i : i + batch_size]  # Select batch
        texts = batch[text_column].tolist()  # Extract text column

        try:
            response = openai.moderations.create(input=texts)
            results = response.model_dump()["results"]
            hateflags = [i["categories"]["hate"] for i in results]

            # Assign predictions directly using df.at
            for idx, flag in zip(batch.index, hateflags):
                df.at[idx, "prediction"] = flag

            for idx, result in zip(batch.index, results):
                df.at[idx, "result"] = result

            time.sleep(2)  # Respect API rate limits

        except Exception as e:
            print(e)
            time.sleep(10)
            continue

    return df


In [3]:
df.loc[df["Transcription"].isna(), "prediction"] = False
mdf = moderate_df(df, text_column="Transcription", batch_size=32)

0-Incompatible indexer with Series
32-64-96-128-160-192-224-256-288-320-352-384-416-448-480-512-544-576-608-640-672-704-736-768-800-832-864-896-928-960-992-1024-1056-1088-1120-1152-1184-1216-1248-1280-1312-1344-1376-1408-1440-1472-1504-1536-1568-1600-1632-1664-1696-1728-1760-1792-1824-1856-1888-1920-1952-1984-

In [2]:
# print each with printing label also
# read the pickle file first
import pickle

# with open("moderated_df.pkl", "wb") as f:
#     pickle.dump(mdf, f)
# read the pickle file first
with open("moderated_df.pkl", "rb") as f:
    mdf = pickle.load(f)
print("nonhate", mdf[mdf["label"] == "nonhate"].prediction.value_counts())
print("explicithate", mdf[mdf["label"] == "explicithate"].prediction.value_counts())
print("implicithate", mdf[mdf["label"] == "implicithate"].prediction.value_counts())

# mdf[mdf['label']=='nonhate'].prediction.value_counts(), mdf[mdf["label"] == "explicithate"].prediction.value_counts(), mdf[mdf["label"] == "implicithate"].prediction.value_counts()

nonhate prediction
False    1000
Name: count, dtype: int64
explicithate prediction
False    480
True      20
Name: count, dtype: int64
implicithate prediction
False    503
True       6
Name: count, dtype: int64


In [10]:
import pickle

with open("moderated_df.pkl", "wb") as f:
    pickle.dump(mdf, f)

In [10]:
# https://claude.ai/chat/1e69b599-0fae-4419-9b1b-2e7be667875b
import pandas as pd

df = pd.read_csv("hate_speech_annotations_highkappa.csv")